In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\system.jl")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\visual_regimes.jl");

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [105]:
using StaticArrays, DifferentialEquations, DynamicalSystems
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; 

In [4]:
time = 1000; tt = 1000; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep)

(alg = Vern9(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true), adaptive = false, dt = 0.001)

In [432]:
U0 = 0.268371462842221 # 0.265;
I0 = -1.75061296075842 # -1.7064049
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = SA[8.34567030569954, 0.738495457778246, 0.438302434813899]
println("last point  pink trajectory:$u0_1")

last point  pink trajectory:[8.34567030569954, 0.738495457778246, 0.438302434813899]


In [433]:
ds = CoupledODEs(TM, u0_1, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.268371462842221, 0.305, 0.3, -1.75061296075842]
 time:          0.0
 state:         [8.34567030569954, 0.738495457778246, 0.438302434813899]


In [434]:
fp, ei, _ = fixedpoints(ds, box, jacob_TM_);

In [435]:
ei

3-element Vector{Vector{ComplexF64}}:
 [-6.480866327844225 + 0.0im, 2.444720202657989 - 13.668264387945497im, 2.444720202657989 + 13.668264387945497im]
 [-8.818115292388175 + 0.0im, -0.30303030302448075 + 0.0im, 7.750189128855439 + 0.0im]
 [-7.20465396728292 - 4.987918126530632im, -7.20465396728292 + 4.987918126530632im, -0.30303030303166734 + 0.0im]

In [436]:
fp

3-dimensional StateSpaceSet{Float64} with 3 points
 8.30415  0.738431  0.437988
 2.02591  0.915042  0.954813
 1.43604  0.938251  0.967584

In [437]:
Jac = jacob_TM_(fp[1], p, 0)
eivecs = eigvecs(Jac)

3×3 SMatrix{3, 3, ComplexF64, 9} with indices SOneTo(3)×SOneTo(3):
   0.998695+0.0im     0.999855-0.0im            0.999855+0.0im
 -0.0496615+0.0im   -0.0142442-0.00912827im   -0.0142442+0.00912827im
   0.011898+0.0im  0.000652042-0.00167355im  0.000652042+0.00167355im

In [441]:
shift = real(eivecs[:, 1])*1e-7 + fp[1]

3-element SVector{3, Float64} with indices SOneTo(3):
 8.304148085514477
 0.7384311228439159
 0.43798820355557727

In [442]:
prob_for = ODEProblem(TM, shift, (0.0, 1500), p) # 
sol_for = solve(prob_for, alg = Vern9(), abstol = 1e-13, reltol=1e-13)

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 5322-element Vector{Float64}:
    0.0
    0.11567484835291644
    0.21923122276020013
    0.3125252080248583
    0.41018060612155427
    0.4986312915784069
    0.5951899839266575
    0.6946227847705022
    0.7928853138927296
    0.8962102242223835
    0.9924739500078495
    1.1210087148225896
    1.2241316183043132
    ⋮
 1497.1503600492529
 1497.4255703132712
 1497.6964607774269
 1497.960562402674
 1498.2522626207908
 1498.5263189618856
 1498.8048614894342
 1499.091078031326
 1499.3456990949937
 1499.6724825626254
 1499.9558944574233
 1500.0
u: 5322-element Vector{SVector{3, Float64}}:
 [8.304148085514477, 0.7384311228439159, 0.43798820355557727]
 [8.30414803283539, 0.7384311254634545, 0.43798820292798146]
 [8.30414800976533, 0.738431126610645, 0.43798820265313454]
 [8.304147998821412, 0.738431127154846, 0.4379882025227534]
 [8.30414799264232, 0.7384311274621075, 0.43798820244913894]
 [8.304147989589318, 0.738

In [443]:
ts_back, tf_back = 1, 300
ts_for, tf_for = 1, length(sol_for)

f = Figure(resolution = (700, 700))
ax = Axis3(f[1, 1], xlabel = "y", ylabel = "x", zlabel = "E")#, show_axis = true)


#scale!(ax.scene, 50, 50, 1)

#lines!(sol_back[3,ts_back:tf_back], sol_back[2,ts_back:tf_back], sol_back[1,ts_back:tf_back],
    #linewidth = 1.0, color = :black)

lines!(sol_for[3,ts_for:tf_for], sol_for[2,ts_for:tf_for], sol_for[1,ts_for:tf_for],
    linewidth = 1.0, color = :deeppink)

scatter!(fp[1][3], fp[1][2], fp[1][1], markersize = 10.0, color = :orange)
scatter!(fp[2][3], fp[2][2], fp[2][1], markersize = 10.0, color = :blue)
scatter!(fp[3][3], fp[3][2], fp[2][1], markersize = 10.0, color = :red)


display(GLMakie.Screen(), f)

GLMakie.Screen(...)